In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
from sklearn.compose import make_column_transformer,ColumnTransformer
from sklearn.pipeline import make_pipeline,Pipeline

In [3]:
train = pd.read_csv('Train_Data.csv')
test = pd.read_csv('Test_Data.csv')

In [4]:
train.head()

,date,campaign,adgroup,ad,impressions,clicks,cost,conversions,revenue
0,01-08-2020,campaign 1,adgroup 1,ad 1,24,6,0.08,0,0.00
1,01-08-2020,campaign 1,adgroup 2,ad 1,1,0,0.00,0,0.00
2,01-08-2020,campaign 1,adgroup 3,ad 1,13,4,0.04,0,0.00
3,01-08-2020,campaign 1,adgroup 4,ad 1,5,4,0.08,0,0.00
4,01-08-2020,campaign 1,adgroup 1,ad 2,247,126,1.29,4,925.71


In [5]:
train.campaign.value_counts()

campaign 1    4571
Name: campaign, dtype: int64

In [6]:
test.campaign.value_counts()

campaign 1    318
Name: campaign, dtype: int64

In [7]:
train.ad.value_counts()

ad 3     617
ad 1     587
ad 2     512
ad 7     461
ad 32    377
        ... 
ad 69      4
ad 39      4
ad 24      4
ad 47      4
ad 51      3
Name: ad, Length: 70, dtype: int64

In [8]:
test.ad.value_counts()

ad 3     44
ad 1     42
ad 7     33
ad 2     30
ad 56    24
ad 72    22
ad 55    19
ad 10    13
ad 73    13
ad 74    12
ad 5     11
ad 71    11
ad 4     11
ad 8     11
ad 6     11
ad 75    11
Name: ad, dtype: int64

In [9]:
train.ad.unique()

array(['ad 1', 'ad 2', 'ad 3', 'ad 4', 'ad 5', 'ad 6', 'ad 7', 'ad 8',
       'ad 9', 'ad 10', 'ad 11', 'ad 12', 'ad 13', 'ad 14', 'ad 15',
       'ad 16', 'ad 17', 'ad 18', 'ad 19', 'ad 20', 'ad 21', 'ad 22',
       'ad 23', 'ad 24', 'ad 25', 'ad 26', 'ad 27', 'ad 28', 'ad 29',
       'ad 30', 'ad 31', 'ad 32', 'ad 33', 'ad 34', 'ad 35', 'ad 36',
       'ad 37', 'ad 38', 'ad 39', 'ad 40', 'ad 41', 'ad 42', 'ad 43',
       'ad 44', 'ad 45', 'ad 46', 'ad 47', 'ad 48', 'ad 49', 'ad 50',
       'ad 51', 'ad 52', 'ad 53', 'ad 54', 'ad 55', 'ad 56', 'ad 57',
       'ad 58', 'ad 59', 'ad 60', 'ad 61', 'ad 62', 'ad 63', 'ad 64',
       'ad 65', 'ad 66', 'ad 67', 'ad 68', 'ad 69', 'ad 70'], dtype=object)

In [10]:
train.adgroup.value_counts()

adgroup 3    1672
adgroup 1    1314
adgroup 4     858
adgroup 2     727
Name: adgroup, dtype: int64

In [11]:
test.adgroup.value_counts()

adgroup 3    111
adgroup 1     90
adgroup 2     63
adgroup 4     54
Name: adgroup, dtype: int64

In [12]:
train.describe()

,impressions,clicks,cost,conversions,revenue
count,4571.000000,4571.000000,4571.000000,4571.000000,4571.000000
mean,92.635747,43.340844,2.800834,1.339094,231.865340
std,198.349737,100.164913,14.361618,5.223922,1091.742763
min,1.000000,0.000000,0.000000,0.000000,0.000000
25%,6.000000,2.000000,0.040000,0.000000,0.000000
50%,16.000000,7.000000,0.210000,0.000000,0.000000
75%,84.000000,38.000000,1.350000,1.000000,92.600000
max,3239.000000,1771.000000,531.250000,94.000000,20515.410000


In [13]:
test.describe()

,cost,impressions,clicks,conversions
count,318.000000,318.000000,318.000000,318.000000
mean,0.381352,67.144654,29.720126,0.805031
std,0.591480,101.078870,45.200014,1.802480
min,0.000000,1.000000,0.000000,0.000000
25%,0.020000,4.000000,2.000000,0.000000
50%,0.105000,20.000000,9.000000,0.000000
75%,0.490000,103.000000,40.750000,1.000000
max,3.530000,589.000000,276.000000,15.000000


In [14]:
train.shape

(4571, 9)

In [15]:
X_train_orig = train.drop(['revenue','campaign'],axis=1)
X_test_orig = test.copy(deep=True)
X_test_orig = X_test_orig.drop('campaign',axis=1)
Y_train_orig = train['revenue']

In [16]:
X_train_orig.head()

,date,adgroup,ad,impressions,clicks,cost,conversions
0,01-08-2020,adgroup 1,ad 1,24,6,0.08,0
1,01-08-2020,adgroup 2,ad 1,1,0,0.00,0
2,01-08-2020,adgroup 3,ad 1,13,4,0.04,0
3,01-08-2020,adgroup 4,ad 1,5,4,0.08,0
4,01-08-2020,adgroup 1,ad 2,247,126,1.29,4


In [17]:
train.isna().sum()

date           0
campaign       0
adgroup        0
ad             0
impressions    0
clicks         0
cost           0
conversions    0
revenue        0
dtype: int64

In [18]:
test.isna().sum()

date           0
campaign       0
adgroup        0
ad             0
cost           0
impressions    0
clicks         0
conversions    0
dtype: int64

In [19]:
X_train_orig['date'].value_counts()

30-10-2020    35
22-12-2020    34
29-10-2020    34
24-12-2020    33
25-12-2020    32
              ..
30-09-2020    10
11-02-2021    10
29-09-2020     8
28-08-2020     8
31-08-2020     8
Name: date, Length: 212, dtype: int64

In [20]:
X_train_orig['date'] = pd.to_datetime(X_train_orig.date)
X_test_orig['date'] = pd.to_datetime(X_test_orig.date)

In [21]:
X_train_orig.dtypes

date           datetime64[ns]
adgroup                object
ad                     object
impressions             int64
clicks                  int64
cost                  float64
conversions             int64
dtype: object

In [22]:
X_train_orig.head()

,date,adgroup,ad,impressions,clicks,cost,conversions
0,2020-01-08,adgroup 1,ad 1,24,6,0.08,0
1,2020-01-08,adgroup 2,ad 1,1,0,0.00,0
2,2020-01-08,adgroup 3,ad 1,13,4,0.04,0
3,2020-01-08,adgroup 4,ad 1,5,4,0.08,0
4,2020-01-08,adgroup 1,ad 2,247,126,1.29,4


In [23]:
X_train_orig['month'] = X_train_orig['date'].dt.month
X_test_orig['month'] = X_test_orig['date'].dt.month
X_train_orig['day'] = X_train_orig['date'].dt.day
X_test_orig['day'] = X_test_orig['date'].dt.day

In [24]:
X_train_orig.head()

,date,adgroup,ad,impressions,clicks,cost,conversions,month,day
0,2020-01-08,adgroup 1,ad 1,24,6,0.08,0,1,8
1,2020-01-08,adgroup 2,ad 1,1,0,0.00,0,1,8
2,2020-01-08,adgroup 3,ad 1,13,4,0.04,0,1,8
3,2020-01-08,adgroup 4,ad 1,5,4,0.08,0,1,8
4,2020-01-08,adgroup 1,ad 2,247,126,1.29,4,1,8


In [25]:
X_test_orig.head()

,date,adgroup,ad,cost,impressions,clicks,conversions,month,day
0,2021-01-03,adgroup 1,ad 1,0.58,121,49,1,1,3
1,2021-01-03,adgroup 3,ad 1,0.17,22,12,0,1,3
2,2021-01-03,adgroup 4,ad 1,0.05,5,3,0,1,3
3,2021-01-03,adgroup 2,ad 1,0.01,2,1,0,1,3
4,2021-01-03,adgroup 2,ad 2,0.01,3,1,0,1,3


In [26]:
X_train = X_train_orig.drop('date',axis=1)
X_test = X_test_orig.drop('date',axis=1)

In [27]:
col_trans = make_column_transformer((OneHotEncoder(), ['adgroup','ad']) , remainder='passthrough')

In [28]:
X_train.dtypes

adgroup         object
ad              object
impressions      int64
clicks           int64
cost           float64
conversions      int64
month            int64
day              int64
dtype: object

In [29]:
X_train.head()

,adgroup,ad,impressions,clicks,cost,conversions,month,day
0,adgroup 1,ad 1,24,6,0.08,0,1,8
1,adgroup 2,ad 1,1,0,0.00,0,1,8
2,adgroup 3,ad 1,13,4,0.04,0,1,8
3,adgroup 4,ad 1,5,4,0.08,0,1,8
4,adgroup 1,ad 2,247,126,1.29,4,1,8


In [30]:
X_test.head()

,adgroup,ad,cost,impressions,clicks,conversions,month,day
0,adgroup 1,ad 1,0.58,121,49,1,1,3
1,adgroup 3,ad 1,0.17,22,12,0,1,3
2,adgroup 4,ad 1,0.05,5,3,0,1,3
3,adgroup 2,ad 1,0.01,2,1,0,1,3
4,adgroup 2,ad 2,0.01,3,1,0,1,3


In [31]:
X_test = X_test[['adgroup','ad','impressions','clicks','cost','conversions','month','day']]

In [32]:
obj_cols = [col for col in X_train.columns if X_train[col].dtype == 'object']
num_cols = [col for col in X_train.columns if X_train[col].dtype != 'object']

In [33]:
print(num_cols)

['impressions', 'clicks', 'cost', 'conversions', 'month', 'day']


In [34]:
num_trans = StandardScaler()
cat_trans = OneHotEncoder(handle_unknown='ignore')
pre_pros = ColumnTransformer(transformers = [ ('num',num_trans,num_cols) , ('cat',cat_trans,obj_cols) ])

In [35]:
linreg_pipe= Pipeline( steps=[ ('preprocessor',pre_pros) , ('classifier',LinearRegression()) ] )

In [36]:
linreg_pipe.fit(X_train,Y_train_orig)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['impressions', 'clicks',
                                                   'cost', 'conversions',
                                                   'month', 'day']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['adgroup', 'ad'])])),
                ('classifier', LinearRegression())])

In [37]:
pred_test = linreg_pipe.predict(X_test)

In [38]:
pred_train = linreg_pipe.predict(X_train)
print(mean_squared_error(Y_train_orig,pred_train))

59756.26653336948


In [39]:
cross_val_score(linreg_pipe,X_train,Y_train_orig).mean()

0.7883381572149261

In [40]:
predictions = pd.DataFrame(columns=["revenue"],data=pred_test)
predictions.to_csv("prediction_results_linreg.csv", index = False)

In [41]:
X_train.head()

,adgroup,ad,impressions,clicks,cost,conversions,month,day
0,adgroup 1,ad 1,24,6,0.08,0,1,8
1,adgroup 2,ad 1,1,0,0.00,0,1,8
2,adgroup 3,ad 1,13,4,0.04,0,1,8
3,adgroup 4,ad 1,5,4,0.08,0,1,8
4,adgroup 1,ad 2,247,126,1.29,4,1,8


In [42]:
from xgboost import XGBRegressor


In [43]:
XGB_pipe= Pipeline( steps=[ ('preprocessor',pre_pros) , ('classifier',XGBRegressor(max_depth=4,learning_rate=0.004,n_estimators=500)) ] )

In [44]:
XGB_pipe.fit(X_train,Y_train_orig)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['impressions', 'clicks',
                                                   'cost', 'conversions',
                                                   'month', 'day']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['adgroup', 'ad'])])),
                ('classifier',
                 XGBRegressor(base_score=0.5, booster='gbtree',
                              colsample_bylevel=1, colsample_bynode=1,
                              colsample_bytree=1, gamma=0, gpu_id=-1,
                              importance_type='gain',
                              interaction_constraints='', learning_rate=0.004,
                              max_delta_step=0, max_depth=4, min_child_weight=1,
   

In [45]:
cross_val_score(XGB_pipe,X_train,Y_train_orig).mean()

0.8919733324150103

In [46]:
pred_test_XGB = XGB_pipe.predict(X_test)
predictions_XGB = pd.DataFrame(columns=["revenue"],data=pred_test_XGB)
predictions_XGB.to_csv("prediction_results_XGB.csv", index = False)

In [47]:
train_set_comparison = train.copy(deep=True)
pred_train_XGB = XGB_pipe.predict(X_train)
predictions_XGB_train = pd.DataFrame(columns=["revenue"],data=pred_train_XGB)
train_set_comparison['prediction'] = predictions_XGB_train['revenue']
train_set_comparison.head()

,date,campaign,adgroup,ad,impressions,clicks,cost,conversions,revenue,prediction
0,01-08-2020,campaign 1,adgroup 1,ad 1,24,6,0.08,0,0.00,0.304052
1,01-08-2020,campaign 1,adgroup 2,ad 1,1,0,0.00,0,0.00,0.304052
2,01-08-2020,campaign 1,adgroup 3,ad 1,13,4,0.04,0,0.00,0.304052
3,01-08-2020,campaign 1,adgroup 4,ad 1,5,4,0.08,0,0.00,0.304052
4,01-08-2020,campaign 1,adgroup 1,ad 2,247,126,1.29,4,925.71,524.103149


In [48]:
train_set_comparison['Error'] = train_set_comparison['revenue'] - train_set_comparison['prediction']
train_set_comparison.head()

,date,campaign,adgroup,ad,impressions,clicks,cost,conversions,revenue,prediction,Error
0,01-08-2020,campaign 1,adgroup 1,ad 1,24,6,0.08,0,0.00,0.304052,-0.304052
1,01-08-2020,campaign 1,adgroup 2,ad 1,1,0,0.00,0,0.00,0.304052,-0.304052
2,01-08-2020,campaign 1,adgroup 3,ad 1,13,4,0.04,0,0.00,0.304052,-0.304052
3,01-08-2020,campaign 1,adgroup 4,ad 1,5,4,0.08,0,0.00,0.304052,-0.304052
4,01-08-2020,campaign 1,adgroup 1,ad 2,247,126,1.29,4,925.71,524.103149,401.606851


In [49]:
train_set_comparison[train_set_comparison.Error > 2500]

,date,campaign,adgroup,ad,impressions,clicks,cost,conversions,revenue,prediction,Error
678,08-09-2020,campaign 1,adgroup 3,ad 18,1356,714,24.51,36,12460.44,9772.419922,2688.020078
687,09-09-2020,campaign 1,adgroup 1,ad 19,2101,1147,24.95,61,18761.60,15075.775391,3685.824609
688,09-09-2020,campaign 1,adgroup 3,ad 19,1361,713,22.75,47,14862.93,11621.271484,3241.658516
701,10-09-2020,campaign 1,adgroup 3,ad 20,2009,1024,35.16,82,20515.41,15075.775391,5439.634609
703,10-09-2020,campaign 1,adgroup 1,ad 20,1657,944,18.33,91,19149.05,15075.775391,4073.274609
1171,08-10-2020,campaign 1,adgroup 1,ad 28,1507,761,41.92,71,18476.84,15014.951172,3461.888828
1204,10-10-2020,campaign 1,adgroup 3,ad 31,1363,809,36.80,85,17863.91,15014.951172,2848.958828
2253,26-11-2020,campaign 1,adgroup 1,ad 43,2151,1068,121.52,82,16324.10,12677.459961,3646.640039
2259,26-11-2020,campaign 1,adgroup 1,ad 44,1895,1031,186.26,56,12089.80,9104.467773,2985.332227
2262,27-11-2020,campaign 1,adgroup 1,ad 43,3239,1771,531.25,94,15299.90,12523.489258,2776.410742


In [50]:
X_train.head()

,adgroup,ad,impressions,clicks,cost,conversions,month,day
0,adgroup 1,ad 1,24,6,0.08,0,1,8
1,adgroup 2,ad 1,1,0,0.00,0,1,8
2,adgroup 3,ad 1,13,4,0.04,0,1,8
3,adgroup 4,ad 1,5,4,0.08,0,1,8
4,adgroup 1,ad 2,247,126,1.29,4,1,8


In [51]:
X_train_nodate = X_train.drop(['month','day'],axis=1)
X_train_nodate.head()

,adgroup,ad,impressions,clicks,cost,conversions
0,adgroup 1,ad 1,24,6,0.08,0
1,adgroup 2,ad 1,1,0,0.00,0
2,adgroup 3,ad 1,13,4,0.04,0
3,adgroup 4,ad 1,5,4,0.08,0
4,adgroup 1,ad 2,247,126,1.29,4


In [52]:
num_cols_new=np.array(['impressions', 'clicks', 'cost', 'conversions'])

In [53]:
num_trans = StandardScaler()
cat_trans = OneHotEncoder(handle_unknown='ignore')
pre_pros = ColumnTransformer(transformers = [ ('num',num_trans,num_cols_new) , ('cat',cat_trans,obj_cols) ])
XGB_pipe= Pipeline( steps=[ ('preprocessor',pre_pros) , ('classifier',XGBRegressor(max_depth=4,learning_rate=0.004,n_estimators=500)) ] )
XGB_pipe.fit(X_train_nodate,Y_train_orig)
cross_val_score(XGB_pipe,X_train_nodate,Y_train_orig).mean()

0.8819847942048554

In [54]:
num_cols=[col for col in X_train.columns if X_train[col].dtype != 'object']
num_cols

['impressions', 'clicks', 'cost', 'conversions', 'month', 'day']

In [55]:
obj_cols

['adgroup', 'ad']

In [56]:
obj_cols_new=obj_cols.copy()

In [57]:
obj_cols_new.remove('ad')
obj_cols

['adgroup', 'ad']

In [58]:
obj_cols_new

['adgroup']

In [59]:
X_train_noad = X_train.drop(['ad'],axis=1)
num_trans = StandardScaler()
cat_trans = OneHotEncoder(handle_unknown='ignore')
pre_pros = ColumnTransformer(transformers = [ ('num',num_trans,num_cols) , ('cat',cat_trans,obj_cols_new) ])
XGB_pipe= Pipeline( steps=[ ('preprocessor',pre_pros) , ('classifier',XGBRegressor(max_depth=4,learning_rate=0.004,n_estimators=500)) ] )
XGB_pipe.fit(X_train_noad,Y_train_orig)
cross_val_score(XGB_pipe,X_train,Y_train_orig).mean()

0.8906507554797635

In [60]:
pred_test_XGB = XGB_pipe.predict(X_test.drop('ad',axis=1))
predictions_XGB = pd.DataFrame(columns=["revenue"],data=pred_test_XGB)
predictions_XGB.to_csv("prediction_results_XGB_new.csv", index = False)

In [61]:
X_train_new,X_test_new,Y_train_new,Y_test_new = train_test_split(X_train,Y_train_orig,test_size=0.2)


In [62]:
XGB_pipe.fit(X_train_new,Y_train_new)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['impressions', 'clicks',
                                                   'cost', 'conversions',
                                                   'month', 'day']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['adgroup'])])),
                ('classifier',
                 XGBRegressor(base_score=0.5, booster='gbtree',
                              colsample_bylevel=1, colsample_bynode=1,
                              colsample_bytree=1, gamma=0, gpu_id=-1,
                              importance_type='gain',
                              interaction_constraints='', learning_rate=0.004,
                              max_delta_step=0, max_depth=4, min_child_weight=1,
         

In [63]:
pred = XGB_pipe.predict(X_train_new)

In [64]:
print(np.sqrt(mean_squared_error(Y_train_new,pred)))

259.93885014086635
